In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np

2022-07-11 17:38:18.630112: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-11 17:38:18.630159: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i)
#         homo_pairs[idx][2] = np.add(homo_pair[2],j)
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
train_homo_pairs = homo_pairs[0:1200]
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = c_homo[0:1200].reshape((len(c_homo[0:1200]),1))
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [5]:
NN = nn.NN()
NN.train(train_homo_pairs,train_c_homo)
NN.test(test_homo_pairs, test_c_homo)

No setting is specified, default setting will be applied.
training step:     0


2022-07-11 17:38:20.372248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-11 17:38:20.372298: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-11 17:38:20.372330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Bai-Group): /proc/driver/nvidia/version does not exist
2022-07-11 17:38:20.372569: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


training step:   100
training step:   200
training step:   300
training step:   400
training step:   500
training step:   600
training step:   700
training step:   800
training step:   900
training step:  1000
training step:  1100
training step:  1200
training step:  1300
training step:  1400
training step:  1500
training step:  1600
training step:  1700
training step:  1800
training step:  1900
training step:  2000
training step:  2100
training step:  2200
training step:  2300
training step:  2400
training step:  2500
training step:  2600
training step:  2700
training step:  2800
training step:  2900
training step:  3000
training step:  3100
training step:  3200
training step:  3300
training step:  3400
training step:  3500
training step:  3600
training step:  3700
training step:  3800
training step:  3900
training step:  4000
training step:  4100
training step:  4200
training step:  4300
training step:  4400
training step:  4500
training step:  4600
training step:  4700
training step

2022-07-11 18:29:15.311899: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


88 (40, 1) (40, 1)
88 (1, 1) (1, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)
88 (40, 1) (40, 1)


<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [6]:
print(NN.loss)

tf.Tensor(
[8.11006714e-07 1.22442611e-06 9.20956063e-06 2.12365976e-05
 3.18772909e-05 3.64143925e-05 3.28442911e-05 2.28117515e-05
 9.77524815e-06 7.82332620e-07 3.40642532e-06 2.46024174e-05
 6.99721932e-05 1.43701225e-04 2.90376811e-05 5.86120059e-08
 8.22463608e-06 9.84118924e-06 3.41779332e-06 4.73439954e-08
 5.11305507e-06 1.24578191e-05 1.24658145e-05 4.65388212e-06
 3.82389977e-08 7.74446926e-06 2.24854703e-05 2.70420005e-05
 1.14520972e-05 1.85434294e-06 9.24108026e-05 2.21839393e-04
 1.24025655e-06 4.14626447e-05 7.05080092e-05 3.82531070e-05
 2.95234190e-06 9.90294393e-06 5.02619005e-05 9.06542700e-05], shape=(40,), dtype=float32)
